In [ ]:
import numpy as np
from math import *
import matplotlib.pyplot as plt
from matplotlib import *
import scipy
import datetime
import os
import sys
import xlrd #a library to read excel file but it won't automatically load the file with "on_demand=True" 

#-------------You have to manually cope the file name to here---------------------------------------------------------------------------------------------------------------

excelName = "HPK3.2_NEU_2.5E15Neq_Analysis.xlsx"
#"HPK_3.1_LGE5_11_10_19 .xlsx"
#"CV_differen_SET_number_W18.xlsx"

#----------------------------------------------------------------------------------------------------------------------------

currentDirectory = os.getcwd()
# check that the file path exists
if not os.path.exists(excelName):
    print('{} not found'.format(excelName))
    input("Press Enter to exit")
    sys.exit()
else:
    print("Opening the file: " + excelName + "   ... please wait")
    print("Here are the excel sheets:")
    
#use xlrd to open the excel file  
xls = xlrd.open_workbook(r'{}'.format(currentDirectory+"/"+excelName), on_demand=True)
allSheetNames = xls.sheet_names()
for i in range(len(allSheetNames)):
    print("(" + str(i) + ") " + allSheetNames[i])
print()
#use input() to get your desired excel sheets, and check whether your input is valid
checkingInput = True
list_mySheetIndex = []
while checkingInput:
    print("Please enter index(es) of the worksheet(s) you want to open")
    str_mySheetIndexes = input("-----Integers only, separated by commas-----")
    mySheetIndexes = []
    number = "" #a dummy variable
    #trying to convert string input to int array
    list_mySheetIndexes = str_mySheetIndexes.split(',')
    count = 0
    for item in list_mySheetIndexes:
        try:
            if int(item)>=0 and int(item)<=len(allSheetNames)-1:
                mySheetIndexes.append(int(item))
                count+=1
            else:print("Index of out bound, please re-enter!")
        except:
            print("Invalid input, please re-enter!")
            pass
    if count == len(list_mySheetIndexes):
        checkingInput = False
        
print("Your list of indexes is " + str(mySheetIndexes))


#=====================Graph Settings=============================================================================
def getSingleIntInput(saved_var, listOfChoices, inputMessage, exceptionMessage, rangeMessage):
    checkingInput = True
    while checkingInput:
        varInput = input(inputMessage)
        try:
            int(varInput)
            if int(varInput) >= 0 and int(varInput) <= len(listOfChoices) -1:
                checkingInput = False
                saved_var = int(varInput)
                return saved_var
            else: 
                print(rangeMessage)
        except ValueError:
            print(exceptionMessage)
            pass
        
#--------------------------------------------------------------------------------------------------------------
def derivative(xArray,yArray):
    if len(xArray) != len(yArray):
        print("Invalid input : your x variable array and y variable array do not have the same length.")
    else:
        dy_dx = []
        for i in range(len(yArray)):
            if i != len(yArray)-1:
                dy_dx.append((yArray[i+1]-yArray[i])/(xArray[i+1]-xArray[i]))
        return dy_dx
    
#--------------------------------------------------------------------------------------------------------------
xyLabels = ["Bias Voltage From LCR [V]", "Current From LCR [A]", "Capacitance [F]", "Resistance [Ohm]", "1/C^2 [1/F^2]","Bias Voltage [V]","Current [A]","C/A [F/cm^2]","Depth [um]","A^2/C^2 [cm^4/F^2]","Derivative [cm^4/V·F^2]","Doping profile [N/cm^3]"]
def cv_plot(xvarArray,yvarArray,marker,labelName,onSamePlot):
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.set_xlabel(xyLabels[xvar])
    ax1.set_ylabel(xyLabels[yvar], color=color)
    ax1.plot(xvarArray,yvarArray,marker, label = labelName, color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.set_yscale(scales[y_scale])
    ax1.set_xscale(scales[x_scale])
    pyplot.gcf().set_size_inches(12, 7, forward=True)
    plt.title(mySheet.name + "      " + xyLabels[yvar] + " vs " + xyLabels[xvar])

    if xvar == 5 and yvar ==11: #for doping vs voltage
        onSamePlot = True
        upperBound = max(xvarArray) - 5
        lowerBound = 20
        new_xArray = []
        new_yArray = []
        for xIndex in range(len(xvarArray)):
            if xvarArray[xIndex] >=lowerBound and  xvarArray[xIndex] <= upperBound and yvarArray[xIndex] >1e11:
                new_xArray.append(xvarArray[xIndex])
                new_yArray.append(yvarArray[xIndex])   
        minDoping = min(new_yArray)
        id_minDoping = new_xArray[new_yArray.index(minDoping)]
        plt.plot(id_minDoping,minDoping,'*',markersize = 10) # add a star of your minimum doping
        
        plt.plot(xvarArray[0:len(xvarArray)-1],derivative(xvarArray,yvarArray))
        plt.plot(xvarArray[0:len(xvarArray)-2],derivative(xvarArray[0:len(xvarArray)-1],derivative(xvarArray,yvarArray)))
        plt.xlim(0,max(xvarArray))
        plt.ylim(1e12,1e17)
        print("******************************************************************")
        print("Your mininum doping is " + str('{:.2e}'.format(minDoping)) +" N/cm^3 at " + str(id_minDoping) + "V")
        print("DOPING VS VOLTAGE FINISHED")
        plt.pause(0.0001)
    if onSamePlot == True:    
        plt.show()
    else:
        plt.legend(loc='best')
    
    if xvar ==5 and yvar == 4:
        onSamePlot = False
        ax2 = ax1.twinx()
        color = 'tab:blue'
        ax2.set_ylabel('First derivative of 1/C^2', color=color)
        ax2.tick_params(axis='y', labelcolor=color)
        ax2.plot(xvarArray[0:len(xvarArray)-1],derivative(xvarArray,yvarArray))# first derivative 1/C^2
        plt.show()
            
    
        
mySheets =[]
for id in mySheetIndexes:
    mySheets.append(xls.sheet_by_index(id))

sameTypeOfSheets = None
TypeOfSheets = ["Enter plot settings manually","Automatically produce the same type of plots"]
print()
print("Plot setting choices:")
for i in range(len(TypeOfSheets)):
    print("(" + str(i) + ") " + TypeOfSheets[i])
sameTypeOfSheets = getSingleIntInput(sameTypeOfSheets,TypeOfSheets,"Choose your plot setting: ", "Invalid input, please re-enter!", "Index out of range!")

mySheetNames = []
for mySheet in mySheets:
    mySheetNames.append(mySheet.name)
    
if sameTypeOfSheets == 0:
    for mySheet in mySheets:
        rw = 0
        totalRows = len(mySheet.col(0))
        for i in range(totalRows):
            if mySheet.cell(i,0).value == 'V' and mySheet.cell(i,1).value == 'I':
                break
            else:
                rw = rw+1
        varName = "" #dummy variable to pass the each variable name
        allVarNames = []
        for name_in_cell in mySheet.row(rw):
            allVarNames.append(name_in_cell.value)
        if len(allVarNames) > 4:
            print()
            print("In the sheet named " + str(mySheet.name) + "    Your analyzed CV file has these parameters: ")
            print("-----if want to choose voltage as x variable, enter index 5 instead of 0 ! -----")
        else:
            print()
            print("In the sheet named " + str(mySheet.name) + "    Your raw CV file has these parameters: ")
        for i in range(len(allVarNames)):
            print("(" + str(i) + ") " + allVarNames[i])


        xvar = None
        yvar = None
        xvar = getSingleIntInput(xvar,allVarNames,"Please choose the index of your x variable: ", "Invalid input, please re-enter!", "Index out of range!")
        yvar = getSingleIntInput(yvar,allVarNames,"Please choose the index of your y variable: ", "Invalid input, please re-enter!", "Index out of range!")

        print()

        scales = ["linear", "log"]

        x_scale = ""
        y_scale = ""
        print()
        print("Choose your scale of x and y axis: ")
        for i in range(len(scales)):
            print("(" + str(i) + ") " + scales[i])

        x_scale = getSingleIntInput(xvar,scales,"Your x axis scale will be: ", "Invalid input, please re-enter!", "Index out of range!")
        y_scale = getSingleIntInput(yvar,scales,"Your y axis scale will be: ", "Invalid input, please re-enter!", "Index out of range!")

        print("Ploting " + allVarNames[yvar] + " vs " + allVarNames[xvar] + "...")
        #start to graph
        xvarArray = []
        yvarArray = []
        for cell in range(rw+1,totalRows):
            xvarArray.append(mySheet.col(xvar)[cell].value)
            yvarArray.append(mySheet.col(yvar)[cell].value)

        cv_plot(xvarArray,yvarArray,',',str(mySheet.name),False)
        
#---------------------------------------------------------------------------------------------------------------

else: 
    xvar = None
    yvar = None
    scales = ["linear", "log"]

    x_scale = ""
    y_scale = ""
    rw = 0
    varName = "" 
    allVarNames = []
    
    for mySheet in mySheets:
        if mySheets.index(mySheet) == 0:
            totalRows = len(mySheet.col(0))
            for i in range(totalRows):
                if mySheet.cell(i,0).value == 'V' and mySheet.cell(i,1).value == 'I':
                    break
                else:
                    rw = rw+1
        
            for name_in_cell in mySheet.row(rw):
                allVarNames.append(name_in_cell.value)
            if len(allVarNames) > 4:
                print()
                print("In " + str(mySheetNames) + "    Your analyzed CV file has these parameters: ")
                print("-----if want to choose voltage as x variable, enter index 5 instead of 0 ! -----")
            else:
                print()
                print("In " + mySheetNames +"    Your raw CV file has these parameters: ")
            for i in range(len(allVarNames)):
                print("(" + str(i) + ") " + allVarNames[i])

            xvar = getSingleIntInput(xvar,allVarNames,"Please choose the index of your x variable: ", "Invalid input, please re-enter!", "Index out of range!")
            yvar = getSingleIntInput(yvar,allVarNames,"Please choose the index of your y variable: ", "Invalid input, please re-enter!", "Index out of range!")

            print()
            print()
            print("Choose your scale of x and y axis: ")
            for i in range(len(scales)):
                print("(" + str(i) + ") " + scales[i])

            x_scale = getSingleIntInput(xvar,scales,"Your x axis scale will be: ", "Invalid input, please re-enter!", "Index out of range!")
            y_scale = getSingleIntInput(yvar,scales,"Your y axis scale will be: ", "Invalid input, please re-enter!", "Index out of range!")

            print("Ploting " + allVarNames[yvar] + " vs " + allVarNames[xvar] + "...")
            print()
            print()
            print()
        else:
            totalRows = len(mySheet.col(0))
            for i in range(totalRows):
                if mySheet.cell(i,0).value == 'V' and mySheet.cell(i,1).value == 'I':
                    break
                else:
                    rw = rw+1
            
            #start to graph
            xvarArray = []
            yvarArray = []
            for cell in range(rw+1,totalRows):
                
                xvarArray.append(mySheet.col(xvar)[cell].value)
                yvarArray.append(mySheet.col(yvar)[cell].value)
            cv_plot(xvarArray,yvarArray,'-',str(mySheet.name),False)
    plt.show()



In [ ]:
#testing inputs are saved here!

#hpk3p2 = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
#hpk3p1(Rene) = [0,1,2,3,4,5,6,7,8,9,10]
#allhpk3p1 = [2,3,4,5,6,7,8,9,10,11,12,13]
#newHpk3p2(Yuchen) = [0,1,2,3,4,5,6]

In [ ]:
#--------------will add the 1-1 correspondence of label names to the variable names in excel---------------
xyLabels = ["Bias Voltage From LCR [V]", "Current From LCR [A]", "Capacitance [F]", "Resistance [Ohm]", "1/C^2 [1/F^2]","Bias Voltage [V]","Current [A]","C/A [F/cm^2]","Depth [um]","A^2/C^2 [cm^4/F^2]","Derivative [cm^4/V·F^2]","Doping profile [N/cm^3]"]

print(labelNames)


In [51]:
#auto analyze raw excel
import xlrd
import os
import xlsxwriter
import math as math
import datetime
import numpy as np


#---------------------------INPUTS------------------------------------------------------------------------------------
existed_Excel_Name = "HPK_3.2_w11_P9_UBM_10kHz.xlsx"
#---------------------------------------------------------------------------------------------------------------------

def getRawValuesInSheet(thisSheet): #this only works for sheet that has v, i ,c, r column
    rw = 0
    varName = "" #dummy variable to pass the each variable name
    rawVarNames = []
    totalRows = len(thisSheet.col(0))
    for i in range(totalRows):
        if thisSheet.cell(i,0).value == 'V' and thisSheet.cell(i,1).value == 'I':
            break
        else:
            rw = rw+1

    for name_in_cell in thisSheet.row(rw):
        rawVarNames.append(name_in_cell.value)
    V = [] 
    I = []
    C = []
    R = []
    for cell in range(rw+1,totalRows):
        V.append(thisSheet.col(0)[cell].value)
        I.append(thisSheet.col(1)[cell].value)
        C.append(thisSheet.col(2)[cell].value)
        R.append(thisSheet.col(3)[cell].value)
    return rawVarNames, V, I, C, R, rw


def makeExcelChart(workbook, sheet, seriesName, chartPosition, chartType, startingCol, endingCol, 
                  xLabel, yLabel, yLog,  chartTitle, xcol, ycol, xlimmin,xlimmax,ylimmin,ylimmax):
    chart = workbook.add_chart({'type': chartType})
    chart.add_series({
    'name': seriesName,
    'categories': '={}!$'.format(seriesName) + '{}$'. format(xcol) +'{}:$'.format(startingCol)+ '{}$'.format(xcol)+'{}'.format(endingCol),
    'values':  '={}!$'.format(seriesName) + '{}$'. format(ycol) +'{}:$'.format(startingCol)+ '{}$'.format(ycol)+'{}'.format(endingCol),
    'marker': {'type': 'circle','size': 3}
    })
    chart.set_x_axis({'name': xLabel, 
                      'major_gridlines': {'visible': True,'line': {'width': 0.25}},
                      'min': xlimmin,
                      'max': xlimmax
                     })
    if yLog == True:
        chart.set_y_axis({'log_base': 10, 'name': yLabel, 
                      'major_gridlines': {'visible': True,'line': {'width': 0.25}},
                      'min': ylimmin,
                      'max': ylimmax
                         })
    else:
        chart.set_y_axis({'name': yLabel, 
                      'major_gridlines': {'visible': True,'line': {'width': 0.25}},
                      'min': ylimmin,
                      'max': ylimmax})
        
    chart.set_title({'name':chartTitle,
                    'num_font':  {'name': 'Arial', 'size': 9,'bold': True}})
    
    chart.set_legend({'position': 'bottom'})
    sheet.insert_chart(chartPosition,chart)
    
def generateAnalysis(excel2Analyze, areaOfSensor): 
    xyLabels = ["Bias Voltage From LCR [V]", "Current From LCR [A]", "Capacitance [F]", "Resistance [Ohm]", "1/C^2 [1/F^2]","Bias Voltage [V]","Current [A]","C/A [F/cm^2]","Depth [um]","A^2/C^2 [cm^4/F^2]","Derivative [cm^4/V·F^2]","Doping profile [N/cm^3]"]

    today = datetime.datetime.today()
    LastProcessed = str(today.month)+ "/" +str(today.day) + "/" + str(today.year) + "_" + str(today.hour)+":"+str(today.minute)+":"+str(today.second)
    if not os.path.exists(excel2Analyze):
        print('{} not found'.format(excel2Analyze))
        input("Press Enter to exit")
        sys.exit()
    else:
        print(">>>Opening the file: " + excel2Analyze + "   ... please wait")
        print("Here are the excel sheets:")
        
        seriesName = existed_Excel_Name[0:existed_Excel_Name.index(".xlsx")]
        #use xlrd to open the excel file  
        rawExcel = xlrd.open_workbook(r'{}'.format(currentDirectory+"/"+excel2Analyze), on_demand=True)
        allSheetNames = rawExcel.sheet_names()
        for i in range(len(allSheetNames)):
            print("(" + str(i) + ") " + allSheetNames[i])
            
        if len(allSheetNames) == 1:
            print(">>>This excel file only has one sheet, trying to get raw data")
            rawSheet = rawExcel.sheet_by_index(0)
        rawData = getRawValuesInSheet(rawSheet)
        
        totalRows = len(rawSheet.col(0))
        if rawData != None:
            print("Getting raw data...done")
        else:
            print("getRawValuesInSheet returns None")
        
        Analyzed_Excel_Name = "auto-analyzed_" + existed_Excel_Name
        
        workbook = xlsxwriter.Workbook(Analyzed_Excel_Name) # create a new workbook
        newSheet = workbook.add_worksheet(seriesName)
        green_cell = workbook.add_format()
        green_cell.set_font_color('green')
        green_cell.set_bg_color('#acff9b')
        green_cell.set_bold()
        cell_format = workbook.add_format()
        cell_format.set_bold()
        
        print(">>>A new excel workbook has been created: " + Analyzed_Excel_Name)
        V_ = getRawValuesInSheet(rawSheet)[1]
        I_ = getRawValuesInSheet(rawSheet)[2]
        C_ = getRawValuesInSheet(rawSheet)[3]
        R_ = getRawValuesInSheet(rawSheet)[4]
        rw = getRawValuesInSheet(rawSheet)[5]
        rawDataList = [V_,I_,C_,R_]
        areaOfSensor = areaOfSensor
        #initializing more columns
        oneOverCsq_ = []
        negV_ = []
        negI_ = []
        CdivA = []
        depth = []
        AsqvCsq =[]
        derivative = []
        doping = []
        newVarList = ['V','I','C','R','1/C^2','-V','-I','C/A','Depth (um)','A^2/C^2 (cm^4/F^2)','Derivative (cm4/V·F²)','N (cm-3)']
        print(">>>Expanding variables..." + str(newVarList) + "...done")
        print(">>>Coping raw data...done")
        newSheet.write('H4',"Area")
        newSheet.write('I4'," = ")
        newSheet.write('J4',areaOfSensor)
        try: 
            str(rawSheet.cell(7,1).value) !=None
            newSheet.write('B8',str(rawSheet.cell(7,1).value))
        except:
            print("[Warning] Cell B8 does not contain frequency of the measurement")
            pass
        #raw data passing to the new worksheet
        for colnum in range(len(newVarList)):
            if colnum == 8 or colnum == 9 or colnum == 10 or colnum == 11:
                newSheet.write(rw,colnum,newVarList[colnum],green_cell)
            else:
                newSheet.write(rw,colnum,newVarList[colnum],cell_format)
                
            for rownum in range(rw+1,totalRows):
                if colnum <4:
                    newSheet.write(rownum,colnum,rawDataList[colnum][rownum-rw-1])
                elif colnum == 4:
                    oneOverCsq_.append(1/math.pow(C_[rownum-rw-1],2))
                    newSheet.write(rownum,colnum, '=1/(C{}^2)'.format(rownum+1)) #1/C^2
                elif colnum == 5:
                    negV_.append(np.abs(V_[rownum-rw-1]))
                    newSheet.write(rownum,colnum,'=ABS(A{})'.format(rownum+1)) #-V
                elif colnum == 6:
                    negI_.append(np.abs(I_[rownum-rw-1]))
                    newSheet.write(rownum,colnum,'=ABS(B{})'.format(rownum+1)) #-I
                elif colnum == 7:
                    CdivA.append(C_[rownum-rw-1]/areaOfSensor)
                    newSheet.write(rownum,colnum,'=C{}'.format(rownum+1)+'/$J$4') #C/A
                elif colnum == 8:
                    depth.append(11.9*0.0000000000000885*10000/C_[rownum-rw-1]*areaOfSensor)
                    newSheet.write(rownum,colnum,'=11.9*0.0000000000000885*10000/C{}*$J$4'.format(rownum+1)) #depth
                elif colnum == 9:
                    AsqvCsq.append(math.pow(areaOfSensor,2)/math.pow(C_[rownum-rw-1],2))
                    newSheet.write(rownum,colnum,'=($J$4/C{})^2'.format(rownum+1)) #A^2/C^2
                elif colnum == 10:
                    if rownum != totalRows -1:
                        derivative.append((AsqvCsq[rownum-rw]-AsqvCsq[rownum-rw-1])/(negV_[rownum-rw]-negV_[rownum-rw-1]))
                    else: 
                        derivative.append('None')
                    newSheet.write(rownum,colnum,'=(J{}'.format(rownum+2) +'-J{})/'.format(rownum+1) +'(F{}-'.format(rownum+2) + 'F{})'.format(rownum+1))
                elif colnum == 11:
                    if rownum != totalRows -1:
                        doping.append(2/(1.6E-19*11.9*0.0000000000000885*derivative[rownum-rw-1]))
                    newSheet.write(rownum,colnum, '=2/(1.6E-19*11.9*0.0000000000000885*K{})'.format(rownum+1))
                    
     
    
        print(">>>Finished analysis")
        
        newSheet.write('A1',existed_Excel_Name[0:existed_Excel_Name.index(".xlsx")])
        newSheet.write('H1',"Last")
        newSheet.write('I1',"Processed")
        newSheet.write('J1', LastProcessed)
        
        #-------graphing Excel charts-------------
        endingNum = len(doping)#doping and derivative has 1 less than voltage
        

        #C vs V
        makeExcelChart(workbook, newSheet, seriesName, 'O3', 'scatter', rw +2, endingNum+rw+2,'Bias Voltage [V]','Capacitance [F]',
                        False, seriesName+ " C vs V",'F','C',min(negV_),max(negV_),0,2e-10)
        makeExcelChart(workbook, newSheet, seriesName, 'W3', 'scatter', rw +2, endingNum+rw+2,'Bias Voltage [V]','Capacitance [F]',
                        True, seriesName+ " C vs V",'F','C',min(negV_),max(negV_),1e-12,2e-10)
        #1/C^2 vs V
        makeExcelChart(workbook, newSheet, seriesName, 'O19', 'scatter', rw +2, endingNum+rw+2,'Bias Voltage [V]','1/C^2 [1/F^2]',
                        True,seriesName+ " 1/C^2 vs V",'F','E',min(negV_),max(negV_),1e19,1e23)
        makeExcelChart(workbook, newSheet, seriesName, 'W19', 'scatter', rw +2, endingNum+rw+2,'Bias Voltage [V]','1/C^2 [1/F^2]',
                        True,seriesName+ " 1/C^2 vs V",'F','E',40,70,1e20,1e23)
        
        #doping vs V
        makeExcelChart(workbook, newSheet, seriesName, 'O35', 'scatter', rw +2,endingNum+rw+2,'Bias Voltage [V]','Doping profile [N/cm^3]',
                        True,seriesName+ " Doping vs V",'F','L',min(negV_),max(negV_),1e12,1e17)
        makeExcelChart(workbook, newSheet, seriesName, 'W35', 'scatter', rw +2,endingNum+rw+2,'Bias Voltage [V]','Doping profile [N/cm^3]',
                        True,seriesName+ " Doping vs V",'F','L',40,70,1e12,1e17)
        #doping vs depth
        makeExcelChart(workbook, newSheet, seriesName, 'O51', 'scatter', rw +2, endingNum+rw+2,'Depth [um]','Doping profile [N/cm^3]',
                        True,seriesName+ " Doping vs Depth",'I','L',0,50,1e12,1e17)
        makeExcelChart(workbook, newSheet, seriesName, 'W51', 'scatter', rw +2, endingNum+rw+2,'Depth [um]','Doping profile [N/cm^3]',
                        True,seriesName+ " Doping vs Depth",'I','L',1,5,1e12,1e17)

        workbook.close()
        print(">>> " + str(Analyzed_Excel_Name) + " has been added to your Jupyter Notebook directory.")
        
        
currentDirectory = os.getcwd()
#must enter a float as the area(the second input), do not enter any non-numerical! 
#Exception-catching will be added later
Analyzed_Excel_Name = generateAnalysis(existed_Excel_Name,0.0169) 


>>>Opening the file: HPK_3.2_w11_P9_UBM_10kHz.xlsx   ... please wait
Here are the excel sheets:
(0) Sheet1
>>>This excel file only has one sheet, trying to get raw data
Getting raw data...done
>>>A new excel workbook has been created: auto-analyzed_HPK_3.2_w11_P9_UBM_10kHz.xlsx
>>>Expanding variables...['V', 'I', 'C', 'R', '1/C^2', '-V', '-I', 'C/A', 'Depth (um)', 'A^2/C^2 (cm^4/F^2)', 'Derivative (cm4/V·F²)', 'N (cm-3)']...done
>>>Coping raw data...done
>>>Finished analysis
>>> auto-analyzed_HPK_3.2_w11_P9_UBM_10kHz.xlsx has been added to your Jupyter Notebook directory.


In [ ]:
xyLabels = ["Bias Voltage From LCR [V]", "Current From LCR [A]", "Capacitance [F]", "Resistance [Ohm]", "1/C^2 [1/F^2]","Bias Voltage [V]","Current [A]","C/A [F/cm^2]","Depth [um]","A^2/C^2 [cm^4/F^2]","Derivative [cm^4/V·F^2]","Doping profile [N/cm^3]"]

HPK_3.2_w11_P9_UBM_10kHz


In [ ]:
import csv

with open('01.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

In [ ]:
#-----------------------------------------some draft codes-------------------------------------------------------------

'''
#practice: extract a number array from a string
string = "2,8,33,ab ,c,90  0, "
numbers = []
number = ""
for char in string:
    if char != ',':
        try:
            int(char)
            if char >= '0' and char <= '9' and char !="":
                number +=char
        except:
            print(char +" is not a number! Invalid input")
    else:
        if len(number)!= 0:
            numbers.append(str(number))
        number = ""
        
print(numbers)

#practice: find character in a string
arr  = range(10)
x = [1,5,7,11]

myname = "heyi"
for char in myname:
    if myname.index(char) == len(myname)-1:
        print("i")
    else:
        print("not i")

#practice: how to use enumerate
print(range(len(allSheetNames)))
numbers  = [17,18,19,20,21,30]
for i in range(len(numbers)):
    if list(enumerate(numbers,start = 0))[i][1] in range(len(allSheetNames)):
        print(str(list(enumerate(numbers,start = 0))[i][1]) + " is in the list")
    else:
        print(str(list(enumerate(numbers,start = 0))[i][1]) + " is not in the list!!!")

'''        

#----------------------------------------Abandoned codes---------------------------------------------------------------------        
'''if len(mySheetIndexes) == 0 or not str_mySheetIndexes:
            print("Your input is empty, please re-enter the indexes of the your desired sheets")
           
elif enumerate() not in range(len(allSheetNames)):
                print("You enter the wrong index: " + str(i) + ". Please only enter the number shown in the list.") 

mySheetIndexes) == 0 or 

for i in mySheetIndex:
    print(i)
mySheets = []
for id in mySheetIndexes:
    mySheet+=xls.sheet_by_index(id)
print(mysheet)'''
#some notes
#xlwt module cannot save .xlsx file, it only can save .xls file which is binary. So I change xlwt to xlsxwritter
'''
wb = xlwt.Workbook(encoding="utf-8")
ws1 = wb.add_sheet('Sheet 1',cell_overwrite_ok=True)
ws2 = wb.add_sheet('Sheet 2',cell_overwrite_ok=True)
ws3 = wb.add_sheet('Sheet 3',cell_overwrite_ok=True)
ws1.row(0).write(0, 'Data written in first cell of first sheet')

ws1.write(0, 0, 'Data overwritten in the first cell of first sheet')

ws2.write(0, 0, 'Data written in first cell of secondsheet')

ws3.write(0, 0, 'Data written in first cell of third sheet')

ws1.write(0, 1, 'Data written in first row,second column offirst sheet')

ws1.row(1).write(1, 'Data written in second row,second column of first sheet')

var = "Data from variable written in third row,second column of first sheet" 

ws1.row(2).write(1,var)
                 
wb.save('Spreadsheet_test2.xls')'''